In [1]:
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!ls /content/drive/MyDrive/input

sample_submission.csv  test.csv		     train_2020-09-16.csv
test_2020-09-16.csv    train_2020-09-09.csv


In [4]:
! pip3 install optuna
! pip3 install catboost
!python -m pip install "dask[complete]"

In [5]:
import numpy as np
import pandas as pd
import gc
import os
import time
import random
from tqdm.auto import tqdm
import datetime
import dask.dataframe as dd

## Read Data

In [6]:
input_type = {
 'customer_id':                 'int64',
 'article_id':                  'int32',
 'label':                       'int8',
 'FN':                          'category',
 'Active':                      'category',
 'club_member_status':          'category',
 'fashion_news_frequency':      'int8',
 'age':                         'int8',
 'a_mean_age':                  'int8',
 'a_std_age':                   'float16',
 'postal_code':                 'category',   
 'product_code':                'category',  
 'product_type_no':             'category',
 'graphical_appearance_no':     'category',
 'colour_group_code':           'category',
 'perceived_colour_value_id':   'category',
 'perceived_colour_master_id':  'category',
 'department_no':               'category',
 'index_code':                  'category',
 'index_group_no':              'category',
 'section_no':                  'category',
 'garment_group_no':            'category',
 'is_for_male':                 'category',
 'is_for_female':               'category',
 'is_for_mama':                 'category',
 'c_mean_price':                'float16',
 'c_std_price':                 'float16',
 'a_mean_price':                'float16',     
 'c_cv_total':                  'int32',
 'a_cv_total':                  'int32', 
 'c_cv_recent':                 'int32',
 'a_cv_recent':                 'int32',
 'a_cv_recent_on':              'int32',
 'a_cv_recent_off':             'int32',
 'a_cv_recent2':                'int32',
 'a_cv_recent_ratio':           'float16',
 'a_cv_recent_ranking':         'int16',
 'a_cv_recent_ranking2':        'int16',
 'c_mean_sales_channel_id':     'float16',
 'a_mean_sales_channel_id':     'float16',
 'c_mean_is_for_male':          'float16',
 'a_mean_is_for_male':          'float16',
 'c_mean_is_for_female':        'float16',
 'a_mean_is_for_female':        'float16',
 'c_mean_is_for_mama':          'float16',
 'diff_dat_last_buy':           'int16',
 'a_mean_is_for_mama':          'float16',
 'ca_index_group_no_num':       'int32',
 'index_group_no_similarity':   'float32',
 'ca_index_code_num':           'int32',
 'index_code_similarity':       'float32',
 'ca_product_group_no_num':     'int32',
 'product_group_no_similarity': 'float32',
 'buy_same_before':             'int16',
 'buy_same_before_recent':      'int16',
 'buy_same_last_week':          'int16',
 'buy_rank1_before':             'int16',
 'buy_rank2_before':             'int16',
 'diff_dat_last_buy_rank1':      'int16',
 'diff_dat_last_buy_rank2':      'int16',
 'delta_mean_price':            'float16',
 'delta_mean_age':              'int8',
 'delta_mean_sales_channel_id': 'float16',
 'delta_mean_is_for_male':      'float16',
 'delta_mean_is_for_female':    'float16',
 'delta_mean_is_for_mama':      'float16',
 'match_rank':                  'int8',
 'als_rank':                    'int8',
 'match_len':                   'int8'
 }

In [7]:
# train data
path = '/content/drive/MyDrive/input'
#train_date_list = ['2020-08-26', '2020-09-02', '2020-09-09', '2020-09-16']
#train_date_list = ['2020-08-26', '2020-09-02', '2020-09-09']
#train_date_list = ['2020-09-09', '2020-09-16']
#train_date_list = ['2020-09-09']
train_date_list = ['2020-09-16']
train_list = [dd.read_csv(f'{path}/train_{date}.csv', dtype=input_type).compute() for date in train_date_list]

/usr/local/lib/python3.7/dist-packages/pandas/core/arrays/categorical.py:2631: FutureWarning: The `inplace` parameter in pandas.Categorical.set_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  res = method(*args, **kwargs)


In [8]:
train = pd.concat([train_list[0]], ignore_index=True)
#train = pd.concat([train_list[0], train_list[1]], ignore_index=True)
#train = pd.concat([train_list[0], train_list[1], train_list[2]], ignore_index=True)
#train = pd.concat([train_list[1], train_list[2]], ignore_index=True)
#train = pd.concat([train_list[2]], ignore_index=True)
#valid = train_list[3].reset_index(drop=True)

In [9]:
# 毎回同じbaselineと比較するため固定
valid = dd.read_csv(f'{path}/train_2020-09-16.csv', dtype=input_type).compute() 
#valid = dd.read_csv(f'./train_2020-09-09.csv', dtype=input_type).compute() 
valid = valid.reset_index(drop=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/arrays/categorical.py:2631: FutureWarning: The `inplace` parameter in pandas.Categorical.set_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  res = method(*args, **kwargs)


In [10]:
train

,customer_id,match_len,article_id,match_rank,als_rank,label,FN,Active,club_member_status,fashion_news_frequency,...,delta_mean_price,delta_mean_age,delta_mean_sales_channel_id,delta_mean_is_for_male,delta_mean_is_for_female,delta_mean_is_for_mama,buy_rank1_before,buy_rank2_before,diff_dat_last_buy_rank1,diff_dat_last_buy_rank2
0,-216791697895142778,10,859105002,0,99,0,False,False,0,2,...,0.010979,-4,0.081055,0.111084,-0.111328,0.0,1,0,78,999
1,-216791697895142778,10,859105009,1,99,0,False,False,0,2,...,0.010094,-1,-0.043945,0.111084,-0.111328,0.0,0,0,999,999
2,-216791697895142778,10,859105004,2,99,0,False,False,0,2,...,0.011894,-2,0.066406,0.111084,-0.111328,0.0,0,0,999,999
3,-216791697895142778,10,859105006,3,99,0,False,False,0,2,...,0.010666,-4,0.146484,0.111084,-0.111328,0.0,7,0,78,999
4,-216791697895142778,10,811907011,4,99,0,False,False,0,2,...,0.006935,0,-0.258789,0.111084,-0.111328,0.0,1,0,81,999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2759355,-8747047206777816159,0,841383002,35,99,0,False,False,0,2,...,-0.008110,-4,-1.806641,0.000000,-1.000000,0.0,0,0,999,999
2759356,-8747047206777816159,0,894780001,36,99,0,False,False,0,2,...,-0.033539,-3,-2.000000,0.000000,-1.000000,0.0,0,0,999,999
2759357,-8747047206777816159,0,706016001,37,99,0,False,False,0,2,...,-0.032440,0,-1.845703,0.000000,-1.000000,0.0,0,0,999,999
2759358,-8747047206777816159,0,936622001,38,99,0,False,False,0,2,...,-0.025070,-5,-2.000000,0.000000,-1.000000,0.0,0,0,999,999


In [11]:
train.columns.values


array(['customer_id', 'match_len', 'article_id', 'match_rank', 'als_rank',
       'label', 'FN', 'Active', 'club_member_status',
       'fashion_news_frequency', 'age', 'postal_code', 'c_mean_price',
       'c_std_price', 'c_cv_total', 'c_cv_recent',
       'c_mean_sales_channel_id', 'c_mean_is_for_male',
       'c_mean_is_for_female', 'c_mean_is_for_mama', 'diff_dat_last_buy',
       'product_code', 'product_type_no', 'graphical_appearance_no',
       'colour_group_code', 'perceived_colour_value_id',
       'perceived_colour_master_id', 'department_no', 'index_code',
       'index_group_no', 'section_no', 'garment_group_no', 'is_for_male',
       'is_for_female', 'is_for_mama', 'product_group_no', 'a_mean_price',
       'a_cv_total', 'a_cv_recent', 'a_cv_recent_on', 'a_cv_recent_off',
       'a_cv_recent2', 'a_cv_recent_ranking', 'a_cv_recent_ranking2',
       'a_cv_recent_ratio', 'a_mean_sales_channel_id', 'a_mean_age',
       'a_std_age', 'ca_index_group_no_num', 'index_group_no_sim

In [12]:
valid

,customer_id,match_len,article_id,match_rank,als_rank,label,FN,Active,club_member_status,fashion_news_frequency,...,delta_mean_price,delta_mean_age,delta_mean_sales_channel_id,delta_mean_is_for_male,delta_mean_is_for_female,delta_mean_is_for_mama,buy_rank1_before,buy_rank2_before,diff_dat_last_buy_rank1,diff_dat_last_buy_rank2
0,-216791697895142778,10,859105002,0,99,0,False,False,0,2,...,0.010979,-4,0.081055,0.111084,-0.111328,0.0,1,0,78,999
1,-216791697895142778,10,859105009,1,99,0,False,False,0,2,...,0.010094,-1,-0.043945,0.111084,-0.111328,0.0,0,0,999,999
2,-216791697895142778,10,859105004,2,99,0,False,False,0,2,...,0.011894,-2,0.066406,0.111084,-0.111328,0.0,0,0,999,999
3,-216791697895142778,10,859105006,3,99,0,False,False,0,2,...,0.010666,-4,0.146484,0.111084,-0.111328,0.0,7,0,78,999
4,-216791697895142778,10,811907011,4,99,0,False,False,0,2,...,0.006935,0,-0.258789,0.111084,-0.111328,0.0,1,0,81,999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2759355,-8747047206777816159,0,841383002,35,99,0,False,False,0,2,...,-0.008110,-4,-1.806641,0.000000,-1.000000,0.0,0,0,999,999
2759356,-8747047206777816159,0,894780001,36,99,0,False,False,0,2,...,-0.033539,-3,-2.000000,0.000000,-1.000000,0.0,0,0,999,999
2759357,-8747047206777816159,0,706016001,37,99,0,False,False,0,2,...,-0.032440,0,-1.845703,0.000000,-1.000000,0.0,0,0,999,999
2759358,-8747047206777816159,0,936622001,38,99,0,False,False,0,2,...,-0.025070,-5,-2.000000,0.000000,-1.000000,0.0,0,0,999,999


In [13]:
del train_list
gc.collect()

50

## Preprocess

In [14]:
train.sort_index(inplace=True)
valid.sort_index(inplace=True)
train.head()

,customer_id,match_len,article_id,match_rank,als_rank,label,FN,Active,club_member_status,fashion_news_frequency,...,delta_mean_price,delta_mean_age,delta_mean_sales_channel_id,delta_mean_is_for_male,delta_mean_is_for_female,delta_mean_is_for_mama,buy_rank1_before,buy_rank2_before,diff_dat_last_buy_rank1,diff_dat_last_buy_rank2
0,-216791697895142778,10,859105002,0,99,0,False,False,0,2,...,0.010979,-4,0.081055,0.111084,-0.111328,0.0,1,0,78,999
1,-216791697895142778,10,859105009,1,99,0,False,False,0,2,...,0.010094,-1,-0.043945,0.111084,-0.111328,0.0,0,0,999,999
2,-216791697895142778,10,859105004,2,99,0,False,False,0,2,...,0.011894,-2,0.066406,0.111084,-0.111328,0.0,0,0,999,999
3,-216791697895142778,10,859105006,3,99,0,False,False,0,2,...,0.010666,-4,0.146484,0.111084,-0.111328,0.0,7,0,78,999
4,-216791697895142778,10,811907011,4,99,0,False,False,0,2,...,0.006935,0,-0.258789,0.111084,-0.111328,0.0,1,0,81,999


In [15]:
# 学習データを特徴量と目的変数に分ける
train_x = train.drop(['label'], axis=1)
train_y = train['label']

valid_x = valid.drop(['label'], axis=1)
valid_y = valid['label']

In [16]:
# rank学習の場合はgroup_idが必要
train_group_id = train_x["customer_id"]
valid_group_id = valid_x["customer_id"]

In [17]:
# 特徴量作成
train_x = train_x.drop(['customer_id', 'article_id'], axis=1)
valid_x = valid_x.drop(['customer_id', 'article_id'], axis=1)

In [18]:
train_x

,match_len,match_rank,als_rank,FN,Active,club_member_status,fashion_news_frequency,age,postal_code,c_mean_price,...,delta_mean_price,delta_mean_age,delta_mean_sales_channel_id,delta_mean_is_for_male,delta_mean_is_for_female,delta_mean_is_for_mama,buy_rank1_before,buy_rank2_before,diff_dat_last_buy_rank1,diff_dat_last_buy_rank2
0,10,0,99,False,False,0,2,31,150330,0.026398,...,0.010979,-4,0.081055,0.111084,-0.111328,0.0,1,0,78,999
1,10,1,99,False,False,0,2,31,150330,0.026398,...,0.010094,-1,-0.043945,0.111084,-0.111328,0.0,0,0,999,999
2,10,2,99,False,False,0,2,31,150330,0.026398,...,0.011894,-2,0.066406,0.111084,-0.111328,0.0,0,0,999,999
3,10,3,99,False,False,0,2,31,150330,0.026398,...,0.010666,-4,0.146484,0.111084,-0.111328,0.0,7,0,78,999
4,10,4,99,False,False,0,2,31,150330,0.026398,...,0.006935,0,-0.258789,0.111084,-0.111328,0.0,1,0,81,999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2759355,0,35,99,False,False,0,2,32,45048,0.000000,...,-0.008110,-4,-1.806641,0.000000,-1.000000,0.0,0,0,999,999
2759356,0,36,99,False,False,0,2,32,45048,0.000000,...,-0.033539,-3,-2.000000,0.000000,-1.000000,0.0,0,0,999,999
2759357,0,37,99,False,False,0,2,32,45048,0.000000,...,-0.032440,0,-1.845703,0.000000,-1.000000,0.0,0,0,999,999
2759358,0,38,99,False,False,0,2,32,45048,0.000000,...,-0.025070,-5,-2.000000,0.000000,-1.000000,0.0,0,0,999,999


In [19]:
categorical_features_indices = np.where(train_x.dtypes == 'category')[0]
categorical_features_indices

array([ 3,  4,  5,  8, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30,
       31])

## Training

In [20]:
from catboost import Pool
import optuna
from catboost import CatBoostRanker, Pool, MetricVisualizer

In [21]:
train_pool = Pool(train_x, train_y, cat_features=categorical_features_indices, group_id=train_group_id)
valid_pool = Pool(valid_x, valid_y, cat_features=categorical_features_indices, group_id=valid_group_id)

In [22]:
train_x.values

array([[10, 0, 99, ..., 0, 78, 999],
       [10, 1, 99, ..., 0, 999, 999],
       [10, 2, 99, ..., 0, 999, 999],
       ...,
       [0, 37, 99, ..., 0, 999, 999],
       [0, 38, 99, ..., 0, 999, 999],
       [0, 39, 99, ..., 0, 999, 999]], dtype=object)

In [23]:
# パラメータの指定, optuna result
params = {
    'loss_function':'YetiRank',
    'iterations' : 290,                         
    'depth' : 8,                                       
    'learning_rate' : 0.2645,               
    'random_strength' : 30,                       
    'bagging_temperature' : 0.05723, 
    'od_wait' :17,
    'od_type': 'Iter',
    'custom_metric': 'MAP:top=12',
    'random_seed': 0,
    'task_type': 'GPU',
    }


In [24]:
"""
def objective(trial):
    # パラメータの指定
    params = {
        'loss_function':'YetiRank',
        'iterations' : trial.suggest_int('iterations', 50, 300),                         
        'depth' : trial.suggest_int('depth', 4, 10),                                       
        'learning_rate' : trial.suggest_loguniform('learning_rate', 0.01, 0.3),               
        'random_strength' :trial.suggest_int('random_strength', 0, 100),                       
        'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 0.01, 100.00), 
        'od_type': trial.suggest_categorical('od_type', ['IncToDec', 'Iter']),
        'od_wait' :trial.suggest_int('od_wait', 10, 50),
        'custom_metric': 'MAP:top=12',
        'task_type': 'GPU',
    }

    model = CatBoostRanker(**params)
    model.fit(train_pool, eval_set=valid_pool, plot=True, early_stopping_rounds=20,)
    return model.get_best_score()['validation']['MAP:top=12']
"""

"\ndef objective(trial):\n    # パラメータの指定\n    params = {\n        'loss_function':'YetiRank',\n        'iterations' : trial.suggest_int('iterations', 50, 300),                         \n        'depth' : trial.suggest_int('depth', 4, 10),                                       \n        'learning_rate' : trial.suggest_loguniform('learning_rate', 0.01, 0.3),               \n        'random_strength' :trial.suggest_int('random_strength', 0, 100),                       \n        'bagging_temperature' :trial.suggest_loguniform('bagging_temperature', 0.01, 100.00), \n        'od_type': trial.suggest_categorical('od_type', ['IncToDec', 'Iter']),\n        'od_wait' :trial.suggest_int('od_wait', 10, 50),\n        'custom_metric': 'MAP:top=12',\n        'task_type': 'GPU',\n    }\n\n    model = CatBoostRanker(**params)\n    model.fit(train_pool, eval_set=valid_pool, plot=True, early_stopping_rounds=20,)\n    return model.get_best_score()['validation']['MAP:top=12']\n"

In [25]:
"""
# optunaで最適値を見つける
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)
"""

"\n# optunaで最適値を見つける\nstudy = optuna.create_study(direction='maximize')\nstudy.optimize(objective, n_trials=100)\n"

In [26]:
model = CatBoostRanker(**params)

In [27]:
model

In [28]:
model.fit(train_pool, 
          eval_set=valid_pool, 
          plot=True,
          early_stopping_rounds=20,
          )

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Groupwise loss function. OneHotMaxSize set to 10


Metric PFound is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric PFound is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric MAP:top=12 is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.0597527	best: 0.0597527 (0)	total: 608ms	remaining: 2m 55s
1:	test: 0.0739783	best: 0.0739783 (1)	total: 1.15s	remaining: 2m 45s
2:	test: 0.0766018	best: 0.0766018 (2)	total: 1.72s	remaining: 2m 44s
3:	test: 0.0775044	best: 0.0775044 (3)	total: 2.26s	remaining: 2m 41s
4:	test: 0.0800801	best: 0.0800801 (4)	total: 2.83s	remaining: 2m 41s
5:	test: 0.0818661	best: 0.0818661 (5)	total: 3.37s	remaining: 2m 39s
6:	test: 0.0827511	best: 0.0827511 (6)	total: 3.92s	remaining: 2m 38s
7:	test: 0.0829022	best: 0.0829022 (7)	total: 4.47s	remaining: 2m 37s
8:	test: 0.0837131	best: 0.0837131 (8)	total: 4.97s	remaining: 2m 35s
9:	test: 0.0836850	best: 0.0837131 (8)	total: 5.53s	remaining: 2m 34s
10:	test: 0.0843649	best: 0.0843649 (10)	total: 6.12s	remaining: 2m 35s
11:	test: 0.0847973	best: 0.0847973 (11)	total: 6.64s	remaining: 2m 33s
12:	test: 0.0848727	best: 0.0848727 (12)	total: 7.11s	remaining: 2m 31s
13:	test: 0.0849910	best: 0.0849910 (13)	total: 7.58s	remaining: 2m 29s
14:	test: 0.

In [29]:
model.get_params()

{'bagging_temperature': 0.05723,
 'custom_metric': 'MAP:top=12',
 'depth': 8,
 'iterations': 290,
 'learning_rate': 0.2645,
 'loss_function': 'YetiRank',
 'od_type': 'Iter',
 'od_wait': 17,
 'random_seed': 0,
 'random_strength': 30,
 'task_type': 'GPU'}

In [30]:
model.get_best_iteration()

289

In [31]:
model.get_best_score()['validation']['MAP:top=12']

0.07183263910475843

In [32]:
feature_importances = model.get_feature_importance(train_pool)
feature_names = train_x.columns
for score, name in sorted(zip(feature_importances, feature_names), reverse=True):
    print('{}: {}'.format(name, score))

diff_dat_last_buy_rank1: 0.012818303143135918
diff_dat_last_buy_rank2: 0.005790495814232743
als_rank: 0.005365485021530235
a_cv_total: 0.005222323520298389
product_group_no_similarity: 0.0035914238588861946
index_code_similarity: 0.0034584553647332983
diff_dat_last_buy: 0.00293942997132135
a_cv_recent_off: 0.002735923581023897
a_cv_recent_ratio: 0.0026275582205194103
buy_same_before: 0.002626044244799101
delta_mean_price: 0.002587372185673509
a_std_age: 0.0025464234141676367
product_code: 0.0022508314913413097
match_len: 0.0020616964610925548
age: 0.0020257315388410024
delta_mean_age: 0.001918552070325097
c_mean_sales_channel_id: 0.0018013347040826966
index_group_no_similarity: 0.0016921240701393797
a_cv_recent: 0.0016408111864156805
a_cv_recent_ranking: 0.0016357990934232625
a_cv_recent2: 0.0015424041540574862
a_mean_price: 0.001511872382829904
a_cv_recent_on: 0.0014531019651805588
delta_mean_sales_channel_id: 0.001410115836206316
ca_product_group_no_num: 0.0013797451312421716
c_std_p

## Test

In [33]:
# メモリ節約
del train, train_x, train_y, valid, valid_x, valid_y, train_pool, valid_pool, train_group_id, valid_group_id
gc.collect()

50

In [34]:
# test data
test = dd.read_csv(f'{path}/test.csv', dtype=input_type).compute()
#test = dd.read_csv(f'{path}/test_2020-09-16.csv', dtype=input_type).compute()
test

/usr/local/lib/python3.7/dist-packages/pandas/core/arrays/categorical.py:2631: FutureWarning: The `inplace` parameter in pandas.Categorical.set_categories is deprecated and will be removed in a future version. Removing unused categories will always return a new Categorical object.
  res = method(*args, **kwargs)


,customer_id,match_len,article_id,match_rank,als_rank,FN,Active,club_member_status,fashion_news_frequency,age,...,delta_mean_price,delta_mean_age,delta_mean_sales_channel_id,delta_mean_is_for_male,delta_mean_is_for_female,delta_mean_is_for_mama,buy_rank1_before,buy_rank2_before,diff_dat_last_buy_rank1,diff_dat_last_buy_rank2
0,-2324495172285268445,6,736530007,0,99,True,False,0,4,21,...,-0.000462,-9,0.287109,0.0,0.0,0.0,0,0,999,999
1,-2324495172285268445,6,770703004,1,99,True,False,0,4,21,...,-0.005836,-12,0.236328,0.0,0.0,0.0,0,0,999,999
2,-2324495172285268445,6,736531006,2,99,True,False,0,4,21,...,0.014046,-10,0.303711,0.0,0.0,0.0,1,0,299,999
3,-2324495172285268445,6,706016001,3,99,True,False,0,4,21,...,-0.008736,-11,0.154297,0.0,0.0,0.0,1,0,299,999
4,-2324495172285268445,6,611415005,4,99,True,False,0,4,21,...,0.007523,-10,0.360352,0.0,0.0,0.0,0,1,999,299
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208755,-3280349465215843523,6,711053003,35,99,False,False,0,2,25,...,0.048035,-8,-0.525391,0.0,0.0,0.0,0,0,999,999
208756,-3280349465215843523,6,904571001,36,99,False,False,0,2,25,...,0.019897,-8,-0.728516,0.0,0.0,0.0,0,0,999,999
208757,-3280349465215843523,6,925404001,37,99,False,False,0,2,25,...,-0.004395,-9,-0.575195,0.0,0.0,0.0,0,0,999,999
208758,-3280349465215843523,6,896170012,38,99,False,False,0,2,25,...,0.044739,-5,0.000000,0.0,0.0,0.0,0,0,999,999


In [35]:
test.sort_index(inplace=True)

In [36]:
test_id = test[['customer_id', 'article_id']]
test = test.drop(['customer_id', 'article_id'], axis=1)

## Prediction

In [37]:
Nrow = 55000000
Ndiv = 10
n = Nrow // Ndiv
test_list = []
for i in range(Ndiv):
  tmp = test.iloc[i*n : (i+1)*n, :]
  test_list.append(tmp)

In [38]:
del test
gc.collect()

50

In [39]:
pred_list = [] 
for i, v in enumerate(test_list):
  print('i=', i)
  tmp = model.predict(v)
  pred_list.append(tmp) 

del test_list
gc.collect()

i= 0
i= 1
i= 2
i= 3
i= 4
i= 5
i= 6
i= 7
i= 8
i= 9


0

In [40]:
pred_list

[array([-1.15485921, -2.20006097, -1.36311772, ..., -1.03821812,
        -0.71960247, -1.2934198 ]),
 array([-2.02391407, -0.89914462, -1.19177784, ..., -0.15503675,
        -0.44516504,  1.40142263]),
 array([-0.17876549, -0.56533994, -0.69113656, ..., -0.67711966,
        -0.39429331, -0.92698835]),
 array([ 0.02677773, -1.3268173 , -1.30216184, ..., -2.20913197,
        -2.50047853, -1.39401931]),
 array([ 0.02204345, -0.74513101, -2.1225384 , ...,  0.24321124,
        -1.72717001, -2.55779648]),
 array([-0.31981383, -0.59354643, -2.2930476 , ..., -0.97385681,
        -1.24410135, -1.36136015]),
 array([-0.88657577, -1.45378047, -0.43123002, ..., -1.05247774,
         1.15416121, -1.02048562]),
 array([-2.10753076, -0.55902576, -0.15106868, ...,  0.02176218,
        -1.56862187,  0.12875126]),
 array([-1.14013996, -0.36446463, -1.62752914, ..., -0.46254996,
        -0.083475  ,  0.04228106]),
 array([-0.48138123, -1.0704963 , -0.87445857, ..., -1.45057529,
        -0.72766183, -1.32

In [41]:
pred = np.concatenate(pred_list)
pred

array([-1.15485921, -2.20006097, -1.36311772, ..., -1.45057529,
       -0.72766183, -1.3276635 ])

In [42]:
del pred_list
gc.collect()

150

In [43]:
test_id = test_id.reset_index(drop=True)
pred = pd.Series(pred, name='score')

In [44]:
sub1 = pd.concat([test_id, pred], axis=1)

In [45]:
sub1 = sub1.sort_values(['customer_id','score'], ascending=False)
sub1 = sub1.groupby('customer_id')['article_id'].apply(list).reset_index(name='prediction')
sub1['prediction'].apply(len)

0          40
1          40
2          40
3          40
4          40
           ..
1371975    40
1371976    40
1371977    40
1371978    40
1371979    40
Name: prediction, Length: 1371980, dtype: int64

In [46]:
sub1['prediction'] = sub1['prediction'].apply(lambda x: x[:12])
sub1['prediction'].apply(len)

0          12
1          12
2          12
3          12
4          12
           ..
1371975    12
1371976    12
1371977    12
1371978    12
1371979    12
Name: prediction, Length: 1371980, dtype: int64

In [47]:
sub1['prediction']

0          [706016001, 673396002, 924243001, 918522001, 7...
1          [910601003, 910601003, 908292002, 903926002, 8...
2          [923758001, 918522001, 610776002, 915529003, 9...
3          [448509014, 706016001, 715624001, 562245046, 9...
4          [751471043, 751471001, 910601003, 924243001, 9...
                                 ...                        
1371975    [706016001, 923758001, 751471001, 714790020, 4...
1371976    [658030020, 658030011, 866731001, 706016001, 6...
1371977    [751471001, 714790020, 923758001, 924243001, 9...
1371978    [924243001, 924243002, 923758001, 930380001, 9...
1371979    [781758003, 781758001, 924243001, 924243002, 9...
Name: prediction, Length: 1371980, dtype: object

In [48]:
# 提出用に元に戻す
sub1['prediction'] = sub1['prediction'].map(lambda x:'0'+' 0'.join(map(str, x)))

In [49]:
sub1


,customer_id,prediction
0,-9223352921020755230,0706016001 0673396002 0924243001 0918522001 07...
1,-9223343869995384291,0910601003 0910601003 0908292002 0903926002 08...
2,-9223321797620987725,0923758001 0918522001 0610776002 0915529003 09...
3,-9223319430705797669,0448509014 0706016001 0715624001 0562245046 09...
4,-9223308614576639426,0751471043 0751471001 0910601003 0924243001 09...
...,...,...
1371975,9223319300843860958,0706016001 0923758001 0751471001 0714790020 04...
1371976,9223333063893176977,0658030020 0658030011 0866731001 0706016001 06...
1371977,9223345314868180224,0751471001 0714790020 0923758001 0924243001 09...
1371978,9223357421094039679,0924243001 0924243002 0923758001 0930380001 09...


## Submit

In [50]:
sub = pd.read_csv(f'{path}/sample_submission.csv')
del sub['prediction']; gc.collect()

200

In [51]:
sub['customer_id2'] = sub['customer_id'].apply(lambda x: int(x[-16:],16) ).astype('int64')

In [52]:
print(sub.shape, sub1.shape)


(1371980, 2) (1371980, 2)


In [53]:
sub1

,customer_id,prediction
0,-9223352921020755230,0706016001 0673396002 0924243001 0918522001 07...
1,-9223343869995384291,0910601003 0910601003 0908292002 0903926002 08...
2,-9223321797620987725,0923758001 0918522001 0610776002 0915529003 09...
3,-9223319430705797669,0448509014 0706016001 0715624001 0562245046 09...
4,-9223308614576639426,0751471043 0751471001 0910601003 0924243001 09...
...,...,...
1371975,9223319300843860958,0706016001 0923758001 0751471001 0714790020 04...
1371976,9223333063893176977,0658030020 0658030011 0866731001 0706016001 06...
1371977,9223345314868180224,0751471001 0714790020 0923758001 0924243001 09...
1371978,9223357421094039679,0924243001 0924243002 0923758001 0930380001 09...


In [54]:
sub =  pd.merge(sub, sub1, left_on='customer_id2', right_on='customer_id', how='left')

In [55]:
sub = sub[['customer_id_x', 'prediction']].copy()
sub.columns = ['customer_id', 'prediction']
print(sub.shape)

(1371980, 2)


In [56]:
sub

,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0568601006 0568601043 0924243001 0915529003 07...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0924243001 0924243002 0915529005 0918522001 09...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0805000001 0794321007 0924243001 0924243002 09...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0923758001 0751471001 0924243002 0915529003 09...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0730683050 0896152002 0791587015 0924243001 07...
...,...,...
1371975,ffffbbf78b6eaac697a8a5dfbfd2bfa8113ee5b403e474...,0557599022 0720125039 0791587001 0611415001 08...
1371976,ffffcd5046a6143d29a04fb8c424ce494a76e5cdf4fab5...,0762846008 0762846031 0448509014 0918522001 07...
1371977,ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...,0762846027 0762846027 0706016002 0762846006 07...
1371978,ffffd7744cebcf3aca44ae7049d2a94b87074c3d4ffe38...,0448509014 0714790020 0924243001 0866731001 07...


In [57]:
#sub = sub[['customer_id', 'valid_pred']].copy()
#sub.columns = ['customer_id', 'prediction']
print(sub.shape)

#sub.to_csv('submission_cat_test.csv', index=False)
sub.to_csv('submission_cat.csv', index=False)

(1371980, 2)


In [58]:
!cp ./submission_cat.csv /content/drive/MyDrive/output/